In [2]:
import pandas as pd
import numpy as np
from subprocess import call
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import pairwise_distances
from surprise import SVD, Dataset, Reader
from scipy import stats

from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import SVDpp, NMF
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans, KNNWithZScore

In [3]:
rating = pd.read_csv("u.data", sep="\t", names = ['user_id', 'item_id', 'rating', 'timestamp'])
rating
rating_matrix1 = rating.pivot_table(index = ['user_id'], columns = ['item_id'], values = ['rating'])
rating_matrix1


rating_matrix = rating_matrix1.dropna(thresh = 10, axis = 1)
rating_matrix = rating_matrix.fillna(0)
Ratings = np.array(rating_matrix)

In [6]:
def Create_Training_set(arr):
    test = arr
    users = []
    items = []
    stars = []
    for i in range(arr.shape[0]):
        a = np.where(test[i] > 0)[0]
        stars.extend(np.take(test[i], a))
        items.extend([k+1 for k in list(a)])
        U = [i+1]*len(a)
        users.extend(U)

    star_dict= {'userID':users,
                'itemsID':items,
                'ratings':stars
                }
    Ratings_for_Svd=pd.DataFrame(star_dict)
    return Ratings_for_Svd

def Create_test_Set(arr):
    test = arr
    users = []
    items = []
    stars = []
    for i in range(arr.shape[0]):
        a = np.where(test[i] == 0)[0]
        stars.extend(np.take(test[i], a))
        items.extend([k+1 for k in list(a)])
        U = [i+1]*len(a)
        users.extend(U)


    star_dict= {'userID':users,
                'itemsID':items,
                'ratings':stars
                }
    Recommendations_for_Svd=pd.DataFrame(star_dict)
    return Recommendations_for_Svd
Ph.shape

(943, 1152)

In [5]:
Ph = pd.read_csv("Pre_use_Predictions.csv")
Ph.drop('Unnamed: 0', inplace=True, axis=1)
Ph = np.array(Ph)

def Make_ready(t, d):
    theta = t
    delta = d
    Augmented_Ratings = np.array(Ratings)
    for i in range(Ratings.shape[0]):
        low = np.percentile(Ph[i], theta)
        vui = np.sum(Ratings[i])/np.sum(Ratings[i] != 0)
        for j in range(Ratings.shape[1]):
            if Ph[i][j] <= low:
                Augmented_Ratings[i][j] = Ratings[i][j] + vui*delta
                
    Ratings_for_Svd = Create_Training_set(Augmented_Ratings)
    Recommendations_for_Svd = Create_test_Set(Augmented_Ratings)
    
    reader=Reader(rating_scale=(0,5))
    Svd_train = Dataset.load_from_df(Ratings_for_Svd[['userID','itemsID','ratings']],reader)

    reader=Reader(rating_scale=(0,5))
    Svd_test = Dataset.load_from_df(Recommendations_for_Svd[['userID','itemsID','ratings']],reader)
    return Svd_train, Svd_test

Svd_train, Svd_test = Make_ready(65, 0.75);    

KeyboardInterrupt: 

In [11]:
Svd_train

In [ ]:
data = Svd_train
param_grid = {'n_epochs': [40], 'lr_all': [0.001, 0.005, 0.01, 0.05, 0.1],
              'reg_all': [0.1, 0.01, 0.001, 0.005]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], n_jobs=-1)

fit = gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
fit